# 3. Feature Engineering

In this notebook, we shall use the data collected from the weather API, on top of the EDA conducted in the previous notebook, to create new features that may be of use for modelling.


In general, there will be two broad categorical features created for the purposes of modelling

* Spatial-based features, where we consider the relationship between the observations made at each station
    * Distance from each station
    * Direction from each station
* Time-based features, where we consider the running the following features
    * Rolling Average or Summed values of weather observations
    * Lagged values, indicating some form of prior values in the model


### 3.1 Imports

As usual, we import the usual packages. We shall also import the data sets for analysis

In [24]:
#import data handling
import numpy as np
import pandas as pd
import itertools
import datetime, time, calendar

#import spatial packages
import pyproj

In [25]:
weather_data = pd.read_csv('../data/00_weatherdata_full.csv')

# station_data_full = pd.read_csv('../data/00_stationdata.csv')

station_data = weather_data.iloc[:,-5:].drop_duplicates().dropna()
station_data.head(2)

,id,name,location,latitude,longitude
0,S100,Woodlands Road,"{'latitude': 1.4172, 'longitude': 103.74855}",1.41720,103.74855
8639,S104,Woodlands Avenue 9,"{'latitude': 1.44387, 'longitude': 103.78538}",1.44387,103.78538


In [27]:
weather_data.head()

,timestamp,station_id,temp,rh,wind_dir,wind_spd,rain,id,name,location,latitude,longitude
0,2020-04-01 00:00:00+08:00,S100,28.2,73.575,69.75,2.475,0.0,S100,Woodlands Road,"{'latitude': 1.4172, 'longitude': 103.74855}",1.4172,103.74855
1,2020-04-01 00:05:00+08:00,S100,28.2,73.640,72.40,2.720,0.0,S100,Woodlands Road,"{'latitude': 1.4172, 'longitude': 103.74855}",1.4172,103.74855
2,2020-04-01 00:10:00+08:00,S100,28.2,73.860,74.00,2.620,0.0,S100,Woodlands Road,"{'latitude': 1.4172, 'longitude': 103.74855}",1.4172,103.74855
3,2020-04-01 00:15:00+08:00,S100,28.2,74.280,73.80,2.560,0.0,S100,Woodlands Road,"{'latitude': 1.4172, 'longitude': 103.74855}",1.4172,103.74855
4,2020-04-01 00:20:00+08:00,S100,28.1,74.600,71.00,2.600,0.0,S100,Woodlands Road,"{'latitude': 1.4172, 'longitude': 103.74855}",1.4172,103.74855


## 3.2 Spatial Features

In this section of the notebook, we shall create a lookup table of the pair-wise relationship of the distance and bearings of each of the 13 stations.

#### 3.2.1 Distance and Bearings Matrix

In this section, we make use of the pairwise combination, along with their respective bearings to create a paiwise distance and bearings matrix.
This will then be appended to the data frame as additional features.

In [76]:
#define function generate pairwise haversine distance matrix
#noted that this is not a significant problem given that the problem is a small case with only 13 x 13 cases.
def pairwise_mat(df, get_dist = True, get_bear = True):
    
    '''
    Custom function to get the pairwise distance and azimuth bearings.    
    '''

    # Prepare for pairwise dictionary generation
    df['longlat_tup'] = df[['longitude','latitude']].apply(tuple, axis = 1)  # Create column with Longitude-latitude tuple
    df = df.loc[:, ['id','longlat_tup',]] # Filter out unnecessary columns

    # parse df as dictionary
    df = df.set_index('id')
    df_as_dict = df.to_dict()
    
    # Create cross_product list, https://www.geeksforgeeks.org/python-all-possible-unique-k-size-combinations-till-n/
    station_id_list = list(df_as_dict['longlat_tup'].keys())
    permut_list = list(itertools.product(station_id_list, station_id_list))
    
    # Initialize
    # Create empty dictionay. To be grown with distance list
    dist_mat, bear_mat = {},{}
    geodesic = pyproj.Geod(ellps='WGS84')

    for station in station_id_list:
        dist_mat[station] = {}
        bear_mat[station] = {}

    # Loop through all permutations, and get distance between points
    for pair in permut_list:
        id_1, id_2 = pair[0], pair[1]
        long1, lat1 = df_as_dict['longlat_tup'][id_1][0], df_as_dict['longlat_tup'][id_1][1]
        long2, lat2 = df_as_dict['longlat_tup'][id_2][0], df_as_dict['longlat_tup'][id_2][1]

        # Get azimuth and Distance between points, https://stackoverflow.com/questions/54873868/python-calculate-bearing-between-two-lat-long
        bear_2_1, bear_1_2, dist_1_2 = geodesic.inv(long1, lat1, long2, lat2)

        # Append to distance dictionary
        dist_mat[id_1][id_2] = round(dist_1_2/1000,3)
        
        # Append to bearings dictionary. Normalize to 360 degress, positive angle.
        if id_1 == id_2:
            bear_mat[id_1][id_2] = np.nan
        else:
            bear_mat[id_1][id_2] = round((bear_2_1 % 360),3)

        # Cast distance dictionary to data_frame
        dist_mat_df = pd.DataFrame(dist_mat).T
        dist_mat_df = dist_mat_df.add_suffix('_distfrom')
        dist_mat_df = dist_mat_df.reset_index()
        dist_mat_df = dist_mat_df.rename({'index': 'id'}, axis = 1)

        # Cast bearing dictionary to data_frame
        bear_mat_df = pd.DataFrame(bear_mat).T
        bear_mat_df = bear_mat_df.add_suffix('_bear')
        bear_mat_df = bear_mat_df.reset_index()
        bear_mat_df = bear_mat_df.rename({'index': 'id'}, axis = 1)
    
    # Export required data
    if get_dist == True and get_bear == True:
        bear_mat_df = bear_mat_df.drop(columns = ['id'])
        return pd.concat([dist_mat_df, bear_mat_df], axis = 1)
    elif get_bear == False:
        return dist_mat_df
    elif get_dist == False:
        return bear_mat_df

In [29]:
pairwise_mat(station_data, get_dist = True, get_bear = True).head()

,id,S100_dist,S104_dist,S106_dist,S107_dist,S108_dist,S109_dist,S111_dist,S115_dist,S116_dist,...,S108_bear,S109_bear,S111_bear,S115_bear,S116_bear,S121_bear,S24_bear,S43_bear,S44_bear,S50_bear
0,S100,0.000,5.049,24.344,26.427,20.349,12.075,15.326,19.899,15.073,...,138.251,111.937,140.307,226.697,177.694,210.665,101.841,118.878,223.329,161.196
1,S104,5.049,0.000,20.465,24.420,20.446,10.301,15.801,24.913,18.345,...,152.469,136.408,158.897,228.227,190.975,221.743,110.967,135.245,226.797,184.482
2,S106,24.344,20.465,0.000,11.435,18.593,13.881,18.706,41.140,28.089,...,215.495,251.229,231.094,250.694,237.686,259.898,162.554,226.137,256.136,246.569
3,S107,26.427,24.420,11.435,0.000,10.913,14.400,14.026,38.354,23.481,...,250.097,298.882,268.669,266.743,261.199,283.811,20.433,289.349,276.528,276.171
4,S108,20.349,20.446,18.593,10.913,0.000,10.926,5.063,28.073,12.944,...,NaN,347.589,312.018,273.135,270.540,301.999,52.127,16.360,289.152,299.758


## 3.3 Temporal Features

Next, we shall create time-based features. In this section, we shall create custom functions to create additional features, and then append them to the data frame in the interest of reducing memory usage. 

Also, for the purposes of clarity, we shall establish clear definitions of rain.

### Definition of Rain

From a meterological perspective, there can be multiple definitions of rain. For the purposes of modelling, we shall define rain to have the following characteristics:
* In a 5-minute lookback window, rain collected by the sensor to be <u>more than 0.0 mm</u>.
* Rain to be <u>sustained for at least 20 minutes</u>. (Continuous rain for at least 20minutes)
    * In this case, 20 minutes was selected on the basis that it takes approximately 20 minutes for a delivery rider to complete an order.
    * This, however, can be considered conservative, and should be adjusted accordingly.

#### 3.3.0 Get Month, Day, Minute columns

In [31]:
def get_datetime(df):
    # DataFrame prep - force timestamp to be datetime format
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['month'] = df['timestamp'].dt.month
    df['day'] = df['timestamp'].dt.day
    df['hour'] = df['timestamp'].dt.hour
    df['minute'] = df['timestamp'].dt.minute
    
    return df

In [32]:
get_datetime(weather_data).head(3)

,timestamp,station_id,temp,rh,wind_dir,wind_spd,rain,id,name,location,latitude,longitude,month,day,hour,minute
0,2020-04-01 00:00:00+08:00,S100,28.2,73.575,69.75,2.475,0.0,S100,Woodlands Road,"{'latitude': 1.4172, 'longitude': 103.74855}",1.4172,103.74855,4,1,0,0
1,2020-04-01 00:05:00+08:00,S100,28.2,73.640,72.40,2.720,0.0,S100,Woodlands Road,"{'latitude': 1.4172, 'longitude': 103.74855}",1.4172,103.74855,4,1,0,5
2,2020-04-01 00:10:00+08:00,S100,28.2,73.860,74.00,2.620,0.0,S100,Woodlands Road,"{'latitude': 1.4172, 'longitude': 103.74855}",1.4172,103.74855,4,1,0,10


#### 3.3.1 Lag all observations, 40 min lookback
In order to get a sense of the previous observations, we shall obtain a dataframe consisting of the lag columns, for up to 40 minutes ago. 

In [83]:
def get_lags(df, lag_period = 8, dropna = False):
    # Force timestamp to datetime
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    
    cols_to_lag = ['temp','rh','wind_dir','wind_spd','rain']
    lags = range(1,lag_period+1)
    
    df = df.assign(**{f'{col}_lag_{str(int(lag*5))}': df.groupby(['station_id'])[col].shift(lag)
                      for lag in lags
                      for col in cols_to_lag
                     })
    if dropna == True:
        df = df[df['temp_lag_'+str(lag_period*5)].notna()]
        
    return df

In [84]:
get_lags(weather_data, lag_period = 2, dropna = True)

,timestamp,station_id,temp,rh,wind_dir,wind_spd,rain,id,name,location,...,temp_lag_5,rh_lag_5,wind_dir_lag_5,wind_spd_lag_5,rain_lag_5,temp_lag_10,rh_lag_10,wind_dir_lag_10,wind_spd_lag_10,rain_lag_10
2,2020-04-01 00:10:00+08:00,S100,28.20,73.86,74.0,2.62,0,S100,Woodlands Road,"{'latitude': 1.4172, 'longitude': 103.74855}",...,28.20,73.64,72.4,2.72,0.0,28.20,73.575,69.75,2.475,0.0
3,2020-04-01 00:15:00+08:00,S100,28.20,74.28,73.8,2.56,0,S100,Woodlands Road,"{'latitude': 1.4172, 'longitude': 103.74855}",...,28.20,73.86,74.0,2.62,0.0,28.20,73.640,72.40,2.720,0.0
4,2020-04-01 00:20:00+08:00,S100,28.10,74.60,71.0,2.60,0,S100,Woodlands Road,"{'latitude': 1.4172, 'longitude': 103.74855}",...,28.20,74.28,73.8,2.56,0.0,28.20,73.860,74.00,2.620,0.0
5,2020-04-01 00:25:00+08:00,S100,28.10,74.90,72.6,2.64,0,S100,Woodlands Road,"{'latitude': 1.4172, 'longitude': 103.74855}",...,28.10,74.60,71.0,2.60,0.0,28.20,74.280,73.80,2.560,0.0
6,2020-04-01 00:30:00+08:00,S100,28.10,75.06,70.2,2.64,0,S100,Woodlands Road,"{'latitude': 1.4172, 'longitude': 103.74855}",...,28.10,74.90,72.6,2.64,0.0,28.10,74.600,71.00,2.600,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3178156,2022-04-30 23:35:00+08:00,S50,26.72,88.16,115.8,1.12,0,S50,Clementi Road,"{'latitude': 1.3337, 'longitude': 103.7768}",...,26.54,89.34,150.6,1.44,0.0,26.46,89.620,167.80,1.080,0.0
3178157,2022-04-30 23:40:00+08:00,S50,26.58,89.42,42.0,0.90,0,S50,Clementi Road,"{'latitude': 1.3337, 'longitude': 103.7768}",...,26.72,88.16,115.8,1.12,0.0,26.54,89.340,150.60,1.440,0.0
3178158,2022-04-30 23:45:00+08:00,S50,26.56,90.24,279.8,1.12,0,S50,Clementi Road,"{'latitude': 1.3337, 'longitude': 103.7768}",...,26.58,89.42,42.0,0.90,0.0,26.72,88.160,115.80,1.120,0.0
3178159,2022-04-30 23:50:00+08:00,S50,26.82,88.96,212.6,1.04,0,S50,Clementi Road,"{'latitude': 1.3337, 'longitude': 103.7768}",...,26.56,90.24,279.8,1.12,0.0,26.58,89.420,42.00,0.900,0.0


#### 3.3.2 Obtaining Prediction Column - 20min Consecutive Rain

In order to get this, the following pseudo code shall be applied
1. Convert rain values from numerical to categorial
    - if rain > 0, return 1
2. Create 4 x time period lead columns for rain variables
    - df.groupby(station_id).shift(-1)
    - df.group(station_id).shift(-2)
    - ...
3. Create column for consective 20min rain:
    - use values (binary 1/0) from the four lead columns to multiply by each other
        - if consecutive rain, 1 * 1 * 1 * 1 = 1
        - if any point indicates no rain, 1 * 1 * 0 * 1 = 0
    - df['consec_rain_20'] = df['rain_lead05'] * df['rain_lead10'] * ... * df['rain_lead20']
4. Drop previous lead columns
    - pd.drop(columns = ['rain_lead05', 'rain_lead10'...])

In [35]:
def get_preds(df, rain_threshold = 0.0, shift_period = 4, dropna = False):
    
    # Force timestamp to datetime
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    
    # Binary Convertor for rain values
    df['rain'] = df['rain'].apply(lambda x: 1 if x > rain_threshold else 0)
    
    
    # Shift rain values, 20 min ahead for prediction, https://stackoverflow.com/questions/48818213/make-multiple-shifted-lagged-columns-in-pandas
    lags = range(-1,-shift_period-1,-1)
    df = df.assign(**{f'rain_lead{-lag*5}': df.groupby(['station_id'])['rain'].shift(lag)
                 for lag in lags 
                })
    if dropna == True:
        df = df.dropna()  

    # Get consecutive rain 
    pred_name = 'rain_consec_'+str(int(shift_period*5))
    df[pred_name] = df.iloc[:,-4:].product(axis = 1)
    df[pred_name] = df[pred_name].map(int)
    
    # Drop lead columns
    col_to_drop = list(df.iloc[:, -shift_period-1:-1].columns)
    df = df.drop(columns = col_to_drop)
    

    return df

In [36]:
get_preds(weather_data, rain_threshold = 0.0, shift_period = 4)

,timestamp,station_id,temp,rh,wind_dir,wind_spd,rain,id,name,location,latitude,longitude,month,day,hour,minute,rain_consec_20
0,2020-04-01 00:00:00+08:00,S100,28.20,73.575,69.75,2.475,0,S100,Woodlands Road,"{'latitude': 1.4172, 'longitude': 103.74855}",1.4172,103.74855,4,1,0,0,0
1,2020-04-01 00:05:00+08:00,S100,28.20,73.640,72.40,2.720,0,S100,Woodlands Road,"{'latitude': 1.4172, 'longitude': 103.74855}",1.4172,103.74855,4,1,0,5,0
2,2020-04-01 00:10:00+08:00,S100,28.20,73.860,74.00,2.620,0,S100,Woodlands Road,"{'latitude': 1.4172, 'longitude': 103.74855}",1.4172,103.74855,4,1,0,10,0
3,2020-04-01 00:15:00+08:00,S100,28.20,74.280,73.80,2.560,0,S100,Woodlands Road,"{'latitude': 1.4172, 'longitude': 103.74855}",1.4172,103.74855,4,1,0,15,0
4,2020-04-01 00:20:00+08:00,S100,28.10,74.600,71.00,2.600,0,S100,Woodlands Road,"{'latitude': 1.4172, 'longitude': 103.74855}",1.4172,103.74855,4,1,0,20,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3178156,2022-04-30 23:35:00+08:00,S50,26.72,88.160,115.80,1.120,0,S50,Clementi Road,"{'latitude': 1.3337, 'longitude': 103.7768}",1.3337,103.77680,4,30,23,35,0
3178157,2022-04-30 23:40:00+08:00,S50,26.58,89.420,42.00,0.900,0,S50,Clementi Road,"{'latitude': 1.3337, 'longitude': 103.7768}",1.3337,103.77680,4,30,23,40,0
3178158,2022-04-30 23:45:00+08:00,S50,26.56,90.240,279.80,1.120,0,S50,Clementi Road,"{'latitude': 1.3337, 'longitude': 103.7768}",1.3337,103.77680,4,30,23,45,0
3178159,2022-04-30 23:50:00+08:00,S50,26.82,88.960,212.60,1.040,0,S50,Clementi Road,"{'latitude': 1.3337, 'longitude': 103.7768}",1.3337,103.77680,4,30,23,50,0


## 3.4 Combining all functions to get final data frame

Finally, we shall make use of the custom functions to obtain a dataframe with all features included. 

The following steps shall be applied 
1. Obtain pivot table of all rain status, islandwide
2. Obtain pairwise distance matrix 
    - Merge with existing weather data
3. Split datetime function into 
    - month
    - day
    - hour
    - minute
4. Obtain Lag columns
    - 8 period lag to be used
5. Conduct one-hot encoding on Station_id column
    - drop duplicated columns
    - drop latitude, longitude data
6. Merge with pivot table of rain status

In [97]:
def add_features(df_weather, df_station):
    
    # get matrix on rain data   
    df_weather_rain_pivot = df_weather.loc[:,['timestamp','station_id','rain']]
    df_weather_rain_pivot = df_weather_rain_pivot.pivot(index = 'timestamp',
                                                columns = 'station_id',
                                                values = 'rain')
    df_weather_rain_pivot = df_weather_rain_pivot.fillna(0)
    df_weather_rain_pivot = df_weather_rain_pivot.add_suffix('_rain')
    for col in df_weather_rain_pivot.columns:
        df_weather_rain_pivot[col] = df_weather_rain_pivot[col].map(int)


    
    #add pairwise distance-bearings matrix
    dist_bear_mat = pairwise_mat(df_station, get_dist = True, get_bear = False)
    df_weather = df_weather.merge(right = dist_bear_mat,
                                  how = 'left',
                                  left_on = 'id',
                                  right_on = 'id')
    

    
    #get date time features
    df_weather = get_datetime(df_weather)
    
    #get lagged features
    df_weather = get_lags(df_weather, lag_period = 8, dropna = True)
    
    #get predictor columns
    df_weather = get_preds(df_weather, rain_threshold = 0.0, shift_period = 4, dropna = False)
    

    # one-hot encoding on station id column
    labels = pd.get_dummies(df_weather.id, prefix = 'station')
    df_weather = pd.concat([df_weather, labels], axis = 1)
    df_weather = df_weather.drop(columns = ['station_id','id','name','location','rain'], axis = 1)
    
    # merge with rain matrix
    df_weather = df_weather.merge(df_weather_rain_pivot,
                                  how = 'left',
                                  left_on = 'timestamp',
                                  right_on = 'timestamp')
        
#     df_weather = df_weather.drop(columns = ['timestamp'])
    
    return df_weather


weather_df_add_feat = add_features(weather_data, station_data)
print(weather_df_add_feat.shape)
weather_df_add_feat.columns

(3170412, 97)


Index(['timestamp', 'temp', 'rh', 'wind_dir', 'wind_spd', 'latitude',
       'longitude', 'month', 'day', 'hour', 'minute', 'S100_distfrom',
       'S104_distfrom', 'S106_distfrom', 'S107_distfrom', 'S108_distfrom',
       'S109_distfrom', 'S111_distfrom', 'S115_distfrom', 'S116_distfrom',
       'S121_distfrom', 'S24_distfrom', 'S43_distfrom', 'S44_distfrom',
       'S50_distfrom', 'temp_lag_5', 'rh_lag_5', 'wind_dir_lag_5',
       'wind_spd_lag_5', 'rain_lag_5', 'temp_lag_10', 'rh_lag_10',
       'wind_dir_lag_10', 'wind_spd_lag_10', 'rain_lag_10', 'temp_lag_15',
       'rh_lag_15', 'wind_dir_lag_15', 'wind_spd_lag_15', 'rain_lag_15',
       'temp_lag_20', 'rh_lag_20', 'wind_dir_lag_20', 'wind_spd_lag_20',
       'rain_lag_20', 'temp_lag_25', 'rh_lag_25', 'wind_dir_lag_25',
       'wind_spd_lag_25', 'rain_lag_25', 'temp_lag_30', 'rh_lag_30',
       'wind_dir_lag_30', 'wind_spd_lag_30', 'rain_lag_30', 'temp_lag_35',
       'rh_lag_35', 'wind_dir_lag_35', 'wind_spd_lag_35', 'rain_lag_3

In [99]:
weather_df_add_feat.to_csv('../data/03_weather_df_add_feat.csv', index = False)

# 3.5 Summary

In this notebook, we engineer some features from the existing data set, and make use of features such as <u>pairwise distances, lag columns, and consecutive periods</u> to obtain additional features for modelling.